# Serve Llama2-70B on SageMaker using DJL container

In this notebook, we deploy the Llama-2-70B model across GPUs on a ml.p4d.24xlarge instance.

## Import the relevant libraries and configure several global variables using boto3

In [ ]:
!pip install sagemaker boto3 --upgrade

In [ ]:
import sagemaker
import jinja2
from sagemaker import image_uris
import boto3
import os
import time
import json
from pathlib import Path

In [17]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
model_bucket = sess.default_bucket()  # bucket to house artifacts
s3_code_prefix = "hf-large-model-djl/code_llama2"  # folder within bucket where code artifact will go

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

jinja_env = jinja2.Environment()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Create SageMaker compatible Model artifact, upload model to S3 and bring your own inference script.

SageMaker Large Model Inference containers can be used to host models without providing your own inference code. This is extremely useful when there is no custom pre-processing of the input data or postprocessing of the model's predictions.

SageMaker needs the model artifacts to be in a Tarball format. In this example, we provide the following files - `serving.properties`.

The tarball is in the following format:

```
code
├──── 
│   └── serving.properties

```

- `serving.properties` is the configuration file that can be used to configure the model server.

In [ ]:
!mkdir -p code_llama2

#### Create serving.properties 
This is a configuration file to indicate to DJL Serving which model parallelization and inference optimization libraries you would like to use. Depending on your need, you can set the appropriate configuration.

Here is a list of settings that we use in this configuration file -
- `engine`: The engine for DJL to use. In this case, we intend to use Accelerate and hence set it to **Python**. 
- `option.model_id`: The model id of a pretrained model hosted inside a model repository on huggingface.co (https://huggingface.co/models). The container uses this model id to download the corresponding model repository on huggingface.co.
- `option.tensor_parallel_degree`: Set to the number of GPU devices over which Accelerate needs to partition the model. This parameter also controls the no of workers per model which will be started up when DJL serving runs. As an example if we have a 8 GPU machine and we are creating 8 partitions then we will have 1 worker per model to serve the requests.

For more details on the configuration options and an exhaustive list, you can refer the documentation - https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-configuration.html.

In [ ]:
# define a variable to contain the s3url of the location that has the model
pretrained_model_location = f"s3://sagemaker-example-files-prod-{region}/models/llama2/"
print(f"Pretrained model will be downloaded from ---- > {pretrained_model_location}")

In the below cell, we leverage [Jinja](https://pypi.org/project/Jinja2/) to create a template for serving.properties. Specifically, we parameterize `option.s3url` so that it can be changed based on the pretrained model location.

In [ ]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_llama2_model/

In [ ]:
%%writefile ./code_llama2/serving.properties
engine=MPI
#option.model_id=TheBloke/Llama-2-70B-fp16
option.s3url=s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_llama2_model/
option.tensor_parallel_degree=8
option.dtype=fp16
option.paged_attention=false
#option.rolling_batch=auto
option.rolling_batch=lmi-dist
option.max_rolling_batch_size=8
option.max_rolling_batch_prefill_tokens=1560
option.model_loading_timeout=3600
option.enable_streaming=true

In [ ]:
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
template = jinja_env.from_string(Path("code_llama2/serving.properties").open().read())
Path("code_llama2/serving.properties").open("w").write(
    template.render(s3url=pretrained_model_location)
)
!pygmentize code_llama2/serving.properties | cat -n

**Image URI for the DJL container is being used here**

In [ ]:
# inference_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/djl-ds:latest"
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

**Create the Tarball and then upload to S3 location**

In [ ]:
!rm -f model.tar.gz
!tar czvf model.tar.gz -C code_llama2 .

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

### To create the end point the steps are:

1. Create the Model using the Image container and the Model Tarball uploaded earlier
2. Create the endpoint config using the following key parameters

    a) Instance Type is ml.p4d.24xlarge

    b) ContainerStartupHealthCheckTimeoutInSeconds is 3600 to ensure health check starts after the model is ready    


3. Create the end point using the endpoint config created

#### Create the Model
Use the image URI for the DJL container and the s3 location to which the tarball was uploaded.

The container downloads the model into the `/tmp` space on the container because SageMaker maps the `/tmp` to the Amazon Elastic Block Store (Amazon EBS) volume that is mounted when we specify the endpoint creation parameter VolumeSizeInGB.

For instances like p4dn, which come pre-built with the volume instance, we can continue to leverage the `/tmp` on the container. The size of this mount is large enough to hold the model.

In [ ]:
from sagemaker.utils import name_from_base

model_name = name_from_base(f"tgi-llama2")
print(model_name)

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={"Image": inference_image_uri,
                      "ModelDataUrl": s3_code_artifact,
                      'Environment': {'MODEL_LOADING_TIMEOUT': '3600'}
                     }
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

In [ ]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.48xlarge",
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
        },
    ],
)
endpoint_config_response

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

### This step can take ~ 20 min or longer so please be patient

In [ ]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

#### Leverage Boto3 to invoke the endpoint. 

This is a generative model so we pass in a Text as a prompt and Model will complete the sentence and return the results.

You can pass a prompt as input to the model. This done by setting `inputs` to a prompt. The model then returns a result for each prompt. The text generation can be configured using appropriate parameters. These `parameters` need to be passed to the endpoint as a dictionary of `kwargs`. Refer this documentation - https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationConfig for more details.

The below code sample illustrates the invocation of the endpoint using a text prompt and also sets some parameters.

In [ ]:
endpoint_name="tgi-llama2-2023-09-26-03-09-02-853-endpoint"

In [18]:
from joblib import Parallel, delayed
import time
prompts = [prompt_live,prompt_live,prompt_live]


def call_endpoint(prompt):
    input = {"inputs": prompt, "parameters": parameters}
    input = json.dumps(input)
    start = time.time()

    response = smr_client.invoke_endpoint(EndpointName=endpoint_name,
                                       ContentType='application/json',
                                       Accept='application/json',
                                       Body=input)
    results = response['Body'].read().decode("utf-8")
    end = time.time()
    process_time=end-start
    print("process time:"+str(int(process_time)))
    ouputJson=json.loads(results)
    print(ouputJson)


results = Parallel(n_jobs=3, prefer='threads', verbose=1)(
    delayed(call_endpoint)(prompt)
    for prompt in prompts
)

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


process time:1
{'generated_text': 'The script should be no longer than 1 minute and 30 seconds, and the host should be able to read it fluently and naturally.'}
process time:1
{'generated_text': 'Please create a script for a live broadcast host to promote the iPhone 14.\n1. The script should be written in Chinese.\n2'}
process time:1
{'generated_text': "The script should be written in a professional manner, using a clear and concise tone to highlight the product's features and advantages.\nThe script"}


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    2.5s finished


In [19]:
%%time
parameters = {
                "max_tokens": 500,
                #"do_sample": True,
                "temperature": 0.5
            }


prompt_live="""Create a captivating product promotion script for an e-commerce live broadcast host based on the following product detailed description.
[The iPhone 14 is the latest phone officially released by Apple on September 8, 2022. It is equipped with a 6.1-inch OLED screen and offers six unique color choices: blue, purple, midnight, starlight, red, and yellow. The phone has an elegant size design, with a length of 146.7mm, a width of 71.5mm, a thickness of 7.8mm, and weighs about 172 grams. In terms of performance, the iPhone 14 is equipped with a powerful Apple A15 bionic chip, which contains a 6-core central processor, including 2 performance cores and 4 efficiency cores, and a 5-core GPU graphics processor. It not only supports practical features such as accident detection and satellite communication but also performs excellently in photography. The rear camera includes a 12-megapixel main lens and a 12-megapixel ultra-wide-angle lens, and the front camera is also 12 megapixels. In addition, the phone also supports photographic technologies such as optical engine, deep fusion technology, smart HDR 4, and portrait mode, ensuring that users can easily capture every beautiful moment.]
The script should include the main features and advantages of the product as well as interactive segments, be written in Chinese, and be concise, interesting, and attractive, ensuring to contain all the elements mentioned above.
"""
prompt1 = """根据以下反引号内的商品详细描述，为电商直播主持人创作一段引人注目的商品推介话术
‘’‘
iPhone 14是苹果公司在2022年9月8日正式发布的最新手机。它配备了一块6.1英寸的OLED屏幕，并提供了六种独特的颜色选择：蓝色、紫色、午夜色、星光色、红色和黄色。手机的尺寸设计优雅，长度为146.7毫米，宽度为71.5毫米，厚度为7.8毫米，重量约为172克。
在性能上，iPhone 14搭载了强大的苹果A15仿生芯片，内部含有6 核中央处理器，有 2 个性能核心和 4 个能效核心，还有5 核GPU图形处理器。。它不仅支持车祸检测和卫星通信等实用功能，而且在拍照方面也表现出色。
后置摄像头包括一个1200万像素的主镜头和一个1200万像素的超广角镜头，前置摄像头也是1200万像素
此外，该手机还支持光像引擎、深度融合技术、智能HDR4和人像模式等摄影技术，确保用户可以轻松捕捉每一个美好瞬间
’‘’
话术中应包括商品的主要特点、优势及互动环节,使用中文撰写，并保持话术简洁、有趣且具吸引力,并确保包含上述要求的所有元素"""

prompt2="""根据以下反引号内的关键词，为电商直播主持人创作一段通用的开场、互动或欢迎话术。请确保话术融入这些关键词，使用中文撰写，内容要简洁、有趣且具吸引力，同时适应广泛的商品和场景。
‘’‘
精选
性价比
品质
日常家居
穿搭
限时折扣
免费赠品
抽奖活动
大品牌合作
独家优惠
’‘’
请使用上述关键词，编写一段具有普遍适用性，适于电商直播开头或互动环节的话术"""


prompt3="""
请根据以下反引号内的商品描述、意图、问题模板和回答模板，为电商直播商品生成一个问答库。要求生成的回答应当有至少一组，最多五组。请确保答案基于商品描述和回答模板生成。如果无法生成回答，表示为“根据已知信息无法生成回答”。格式应如下：{{"Q":"问题","A":['答案1-1','答案1-2'...]}}
‘’‘
商品描述：iPhone 14是苹果公司在2022年9月8日正式发布的最新手机。它配备了一块6.1英寸的OLED屏幕，并提供了六种独特的颜色选择：蓝色、紫色、午夜色、星光色、红色和黄色。手机的尺寸设计优雅，长度为146.7毫米，宽度为71.5毫米，厚度为7.8毫米，重量约为172克。
在性能上，iPhone 14搭载了强大的苹果A15仿生芯片，内部含有6 核中央处理器，有 2 个性能核心和 4 个能效核心，还有5 核GPU图形处理器。。它不仅支持车祸检测和卫星通信等实用功能，而且在拍照方面也表现出色。后置摄像头包括一个1200万像素的主镜头和一个1200万像素的超广角镜头，前置摄像头也是1200万像素。此外，该手机还支持光像引擎、深度融合技术、智能HDR4和人像模式等摄影技术，确保用户可以轻松捕捉每一个美好瞬间。}
意图：性能
问题模板：手机的性能如何？
回答模板：[商品名称]采用了最新的[芯片名称]，搭载了[核心数量]核CPU和[GPU核心数量]核GPU，为用户提供强大的性能。
谈到[商品名称]的性能，不得不提及它的[芯片名称]，配备[核心数量]核CPU和[GPU核心数量]核GPU，应对各种任务都游刃有余。
[商品名称]在性能上表现卓越，得益于其[芯片名称]和[核心数量]核处理器，加上[GPU核心数量]核GPU，让每次使用都顺畅无比。}
’‘’
问答生成：请基于上述商品描述、意图、问题模板和回答模板，为电商直播商品提供符合上述格式的问答库。
"""

prompt4="""
请以电商直播主持人的第一人称角度回答观众的商品相关问题。确保只回答与商品相关的问题，并只使用以下反引号内知识库的信息来回答。回答中请勿随意编造内容。格式应如下:[{{"intention": "意图1", "answer": "回答1"}},{{"intention": "意图2", "answer": "回答2"}}]
‘’‘
[问题：iPhone 14有哪些可选的颜色？][回答：iPhone 14提供了六种时尚的颜色选择，包括蓝色、紫色、午夜色、星光色、红色和黄色。][意图：颜色]
[问题：关于摄像头，iPhone 14的前置和后置摄像头分辨率是多少？][回答：iPhone 14的前置和后置摄像头分辨率都是1200万像素。][意图：分辨率]
[问题：我经常用手机办公和玩游戏，iPhone 14的性能如何？][回答：iPhone 14搭载了强大的苹果A15六核中央处理器，无论是玩游戏、看视频，还是办公，它都可以轻松应对。][意图：性能]}
’‘’
观众问题：主播小姐姐好漂亮
使用第一人称直接回答观众关于商品的提问。检查知识库中是否有与观众提问相匹配的回答。对于在知识库中找到的每个匹配意图，请依次提供对应的回答，并确保从知识库中的意图中提取相应的意图标签。如果所有的意图都在知识库中找不到答案，回答“根据已知信息无法回答问题”。确保不使用emoji。
"""

other="""我很在意手机的颜色和摄像头功能，能给我介绍一下iPhone 14在这两方面的特点吗？
便宜点就好了"""
#for i in range(1):
#  response_model = smr_client.invoke_endpoint(
#    EndpointName=endpoint_name,
#    Body=json.dumps(
#        {
#            "inputs": prompt_live,
#            "parameters": parameters,
#        }
#    ),
#    ContentType="application/json",
#  )
#  ouputJson=json.loads(response_model["Body"].read().decode("utf-8"))
#  print("begin=="+str(i)+"============")
#  print(ouputJson["generated_text"])



CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 17.4 µs


In [22]:
import io
import re

def extract_unicode_chars(text):
    pattern = r'\\u([\dA-Fa-f]{4})'
    result = re.sub(pattern, lambda m: chr(int(m.group(1), 16)), text)
    return result


class StreamScanner:
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0
        
        
class LineIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord('\n'):
                self.read_pos += len(line)
                return line[:-1]
            try:
                chunk = next(self.byte_iterator)
            except StopIteration:
                if self.read_pos < self.buffer.getbuffer().nbytes:
                    continue
                raise
            if 'PayloadPart' not in chunk:
                print('Unknown event type:' + chunk)
                continue
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk['PayloadPart']['Bytes'])

In [21]:
from joblib import Parallel, delayed
prompts = [prompt1,prompt2,prompt3,prompt4]


response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt1,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
for event in event_stream:
    eventJson=event['PayloadPart']['Bytes'].decode('utf-8')
    output=(eventJson)
    print(output)
    
#for line in LineIterator(event_stream):
#        data = json.loads(line)
#        output += data['generated_text'][0]+''
#        print(output)  


def call_endpoint(prompt):
    response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

    event_stream = response_model['Body']
    for event in event_stream:
        eventJson=event['PayloadPart']['Bytes'].decode('utf-8')
        output=extract_unicode_chars(eventJson)
        print(output)
    #for line in LineIterator(event_stream):
    #        data = json.loads(line)
    #        output += data['generated_text'][0]+''
    #        print(output)



results = Parallel(n_jobs=10, prefer='threads', verbose=1, )(
    delayed(call_endpoint)(prompt)
    for prompt in prompts
)

{"generated_text": ",
\u5e76
\u4e14
\u4e0d
\u8d85
\u8fc7
5
\u5206
\u949f
\n
\u672c
\u6b21
\u62db
\u8058
\u8981
\u6c42
\u6295
\u7a3f"}


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


{"generated_text": ",
\u5e76
{"generated_text": "\u3002
{"generated_text": "\n
\n
{"generated_text": "\n
##
\n
\n
##
 
\u9898
\u76ee
2
\n
\n
\u4e14
\u4e0d
\u5f97
\u8d85
\u8fc7
3
\u5206
B
\n
\n
\u8bf7
\u6839
\u636e
\u4ee5
\u4e0b
\u5546
\u54c1
\u63cf
\u8ff0
\uff0c
\u751f
\u6210
\u4e00
\u4e2a"}
\u949f
,
\u8bf7
\u5728
\u6587
\u4e2d
\u6ce8
\u660e
\u8d27
\u53f7
,
\u4ef7
\u683c"}
rand
:
 
\u54ea
\u6015
\u662f
\u51ac
\u5929
\uff0c
\u4e5f
\u4e0d
\u59a8
\u788d
\u6211
\u4eec
"}
#
 
\u8bc4
\u4f30
\u6807
\u51c6
\n
\n
-
 
\u56de
\u7b54
\u4e2d
\u81f3
\u5c11
\u6709
\u4e00
\u4e2a"}


[Parallel(n_jobs=10)]: Done   4 out of   4 | elapsed:    6.3s finished


## Conclusion
In this post, we demonstrated how to use SageMaker large model inference containers to host Llama2-70B. For more details about Amazon SageMaker and its large model inference capabilities, refer to the following:

* Model parallelism and large model inference on Sagemaker (https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-large-model-inference.html)

## Clean Up

In [ ]:
# Delete the endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# In case the endpoint failed we still want to delete the model
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)